Problem 10: Machine learning the properties of a particle in a harmonic potential

Use deep learning (DL) to extract the spring constant and the friction coefficient of a noisy oscillator from particle-displacement trajectories x(t), which you can generate using the code from
the previous assignment. The idea is similar to well-established image processing algorithms
based on convolutional neural networks.
First, generate a number of independent particle-displacement trajectories x(t) for different
values of ks and γ. You can simply choose them randomly from the intervals ks ∈ [10, 100] and
γ ∈ [10, 100]. It is advisable to save these trajectories into a file, because you can use them
later as training and test sets (e.g., as 70% and 30% fractions of the data) for your DL model.
No need to save every time step of x(t), for example, every 10 time steps would be sufficient.
Second, build a DL model by using ”keras” module from tensorflow with python implementation. Below is an example of such a model with two convolutional, two max-pooling, and one
dense (fully connected) layers.


model = keras.Sequential()

model.add(layers.Conv1D(filters=*, kernel size=*, strides = *, activation=’relu’, input shape=(*,*)))

model.add(layers.MaxPooling1D(pool size=*))

model.add(layers.Conv1D(filters=*, kernel size=*, strides = *, activation=’relu’))

model.add(layers.MaxPooling1D(pool size=*))

model.add(layers.Flatten())

model.add(layers.Dense(*, activation=’relu’))

model.add(layers.Dense(2, activation=’relu’))

In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.optimize import curve_fit
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

def create_particle(x0,particle_list):
    particle_list.append(x0)

def force(gamma,ks,x,x0,v,dt):
    xi = np.random.normal()
    return -gamma*v-ks*(x-x0)+2*np.sqrt(gamma)*xi/np.sqrt(dt)

def velocity_verlet(x,v,dt,m,gamma,ks,x0):
    f_t = force(gamma,ks,x,x0,v,dt)
    v_half = v+dt/2*f_t/m
    x_next = x+dt*v_half
    f_next = force(gamma,ks,x_next,x0,v_half,dt)
    v_next = v_half+dt/2*f_next/m
    return x_next,v_next

In [48]:
x0, x, v = 0, 0, 0
dt, m = 0.01, 1
sim_time = 100
data_num = 70

for num in range(data_num):
    gamma,ks = np.random.uniform(10,100),np.random.uniform(10,100)
    data = []
    traj_list = []
    for i in range(int(sim_time / dt)):
        if i % 10 == 0:  # 每10个时间步长
            traj_list.append(x)
        x, v = velocity_verlet(x, v, dt, m, gamma, ks, x0)

    time_list = np.arange(0, 100, 0.1)

    # 添加 gamma 和 ks 到 data 的第一行
    data.append([gamma, ks])
    #plt.plot(time_list, traj_list, '.', label=r'$k_s$=' + str(ks))
    data=np.vstack((data,np.column_stack((time_list, traj_list))))
    np.savetxt('./set8/trainning/'+str(num)+'.dat', data, header='gamma\tks', delimiter='\t', fmt='%10.5f')

In [49]:
x0, x, v = 0, 0, 0
dt, m = 0.01, 1
sim_time = 100
data_num = 30

for num in range(data_num):
    gamma,ks = np.random.uniform(10,100),np.random.uniform(10,100)
    data = []
    traj_list = []
    for i in range(int(sim_time / dt)):
        if i % 10 == 0:  # 每10个时间步长
            traj_list.append(x)
        x, v = velocity_verlet(x, v, dt, m, gamma, ks, x0)

    time_list = np.arange(0, 100, 0.1)

    # 添加 gamma 和 ks 到 data 的第一行
    data.append([gamma, ks])
    #plt.plot(time_list, traj_list, '.', label=r'$k_s$=' + str(ks))
    data=np.vstack((data,np.column_stack((time_list, traj_list))))
    np.savetxt('./set8/test/'+str(num)+'.dat', data, header='gamma\tks', delimiter='\t', fmt='%10.5f')

In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Rest of your code...

# Split trajectories into training and test sets
train_fraction = 0.7
train_size = int(train_fraction * num_trajectories)
train_set = trajectories[:train_size]
test_set = trajectories[train_size:]

# Pad the trajectories to have the same length
train_set = pad_sequences(train_set, padding='post')
test_set = pad_sequences(test_set, padding='post')

# Build the DL model
model = tf.keras.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu', input_shape=(train_set.shape[1], 1)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_set, train_set, epochs=10, batch_size=32)

# Evaluate the model
loss = model.evaluate(test_set, test_set)
print('Test Loss:', loss)


In [5]:
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

def force(gamma,ks,x,x0,v,dt):
    xi = np.random.normal()
    return -gamma*v-ks*(x-x0)+2*np.sqrt(gamma)*xi/np.sqrt(dt)

def velocity_verlet(x,v,dt,m,gamma,ks,x0):
    f_t = force(gamma,ks,x,x0,v,dt)
    v_half = v+dt/2*f_t/m
    x_next = x+dt*v_half
    f_next = force(gamma,ks,x_next,x0,v_half,dt)
    v_next = v_half+dt/2*f_next/m
    return x_next,v_next

x0, x, v = 0, 0, 0
dt, m = 0.01, 1

# Generate particle-displacement trajectories
def generate_trajectories(num_trajectories, num_time_steps):
    trajectories = []
    ks_values = np.random.uniform(10, 100, num_trajectories)
    gamma_values = np.random.uniform(10, 100, num_trajectories)
    x0, x, v = 0, 0, 0
    dt, m = 0.01, 1

    for i in range(num_trajectories):
        ks = ks_values[i]
        gamma = gamma_values[i]
        trajectory = []

        for t in range(int(num_time_steps/dt)):
            # Generate random particle displacement
            if i%10 == 0:
                trajectory.append(x)
            x, v = velocity_verlet(x, v, dt, m, gamma, ks, x0)

        trajectories.append(trajectory)

    return np.array(trajectories, dtype=object)

# Generate particle-displacement trajectories
num_trajectories = 100
sim_time = 100
num_time_steps = int(sim_time/dt)
trajectories = generate_trajectories(num_trajectories, sim_time)
# Save trajectories to a file
np.save('trajectories.npy', trajectories)

# Split trajectories into training and test sets
train_fraction = 0.7
train_size = int(train_fraction * num_trajectories)
train_set = trajectories[:train_size]
test_set = trajectories[train_size:]
print(train_set.shape[0])
# Build the DL model
max_trajectory_length = max(len(trajectory) for trajectory in trajectories)
padded_trajectories = pad_sequences(trajectories, maxlen=max_trajectory_length, padding='post', dtype=np.float32)
train_set = padded_trajectories[:train_size]
test_set = padded_trajectories[train_size:]

# Reshape the data for compatibility with Conv1D input shape
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1], 1)
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1], 1)

# Build the DL model
model = tf.keras.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu', input_shape=(train_set.shape[1], 1)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='relu'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_set[1], train_set[0], epochs=10, batch_size=32)

# Evaluate the model
loss = model.evaluate(test_set, test_set)
print('Test Loss:', loss)





70
Epoch 1/10


ValueError: in user code:

    File "c:\Users\Xiongxiao Wang\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Xiongxiao Wang\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Xiongxiao Wang\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Xiongxiao Wang\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Xiongxiao Wang\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return 

In [4]:
predictions = model.predict(test_set)
test_set[0]
predictions[0]

1/1 [==============================] - 0s 79ms/step


array([0.], dtype=float32)